# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path ="../output_data/cities.csv"

cities_df = pd.read_csv(csv_path, encoding="utf-8")
cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.80,-68.30,59.00,51,75,19.46,AR,1604273163
1,Severo-Kuril'sk,50.68,156.12,42.73,71,1,8.97,RU,1604273015
2,Quelimane,-17.88,36.89,71.60,94,20,2.04,MZ,1604273164
3,Punta Arenas,-53.15,-70.92,53.60,76,0,8.05,CL,1604273164
4,Vaini,-21.20,-175.20,78.80,78,75,10.29,TO,1604273164


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = cities_df[["Lat","Lng"]].astype(float)
humidity = cities_df['Humidity'].astype(float)

In [4]:
pprint(gmaps.configure(api_key=g_key))

None


In [5]:

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity, 
    dissipating=False,
    max_intensity=100,
    point_radius = 1
)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_weather = cities_df.loc[(cities_df['Cloudiness'] == 0) & (cities_df['Max Temp'] > 70) & (cities_df['Max Temp'] < 80) & (cities_df['Wind Speed']<10)].dropna()
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,Mae Chan,20.15,99.85,71.60,100,0,1.23,TH,1604273166
53,Bilma,18.69,12.92,70.45,25,0,6.22,NE,1604273017
114,Maceió,-9.67,-35.74,77.00,88,0,9.17,BR,1604273195
124,Salalah,17.02,54.09,73.40,46,0,4.70,OM,1604273197
137,Naliya,23.27,68.83,77.40,40,0,7.36,IN,1604273202
186,Doha,25.29,51.53,77.00,61,0,3.36,QA,1604273066
196,Hervey Bay,-25.30,152.85,79.00,72,0,7.00,AU,1604273219
316,Los Llanos de Aridane,28.66,-17.92,73.40,28,0,8.05,ES,1604272967
359,Bocaiúva,-17.11,-43.81,71.60,83,0,6.93,BR,1604273270
379,Dzaoudzi,-12.79,45.27,73.40,88,0,3.36,YT,1604273278


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
ideal_weather["Hotel Name"] = ""
hotel_df = ideal_weather
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
12,Mae Chan,20.15,99.85,71.60,100,0,1.23,TH,1604273166,
53,Bilma,18.69,12.92,70.45,25,0,6.22,NE,1604273017,
114,Maceió,-9.67,-35.74,77.00,88,0,9.17,BR,1604273195,
124,Salalah,17.02,54.09,73.40,46,0,4.70,OM,1604273197,
137,Naliya,23.27,68.83,77.40,40,0,7.36,IN,1604273202,
186,Doha,25.29,51.53,77.00,61,0,3.36,QA,1604273066,
196,Hervey Bay,-25.30,152.85,79.00,72,0,7.00,AU,1604273219,
316,Los Llanos de Aridane,28.66,-17.92,73.40,28,0,8.05,ES,1604272967,
359,Bocaiúva,-17.11,-43.81,71.60,83,0,6.93,BR,1604273270,
379,Dzaoudzi,-12.79,45.27,73.40,88,0,3.36,YT,1604273278,


In [33]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "Hotel",
    "key": g_key}

# Use the lat/lng we recovered to identify cities
for index, row in ideal_weather.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
           hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
       
    except (KeyError, IndexError):
            print("Missing field/result... skipping.")

Missing field/result... skipping.


In [30]:
Hotel_df= pd.DataFrame(hotel_df, columns=["City", "Lat", "Lng", "Hotel Name"])
Hotel_df.drop
Hotel_df

,City,Lat,Lng,Hotel Name
12,Mae Chan,20.15,99.85,Rimchan Resort Hotel
53,Bilma,18.69,12.92,
114,Maceió,-9.67,-35.74,Jatiúca Hotel & Resort
124,Salalah,17.02,54.09,HYATT SALALAH HOTEL
137,Naliya,23.27,68.83,Hotel Panchvati
186,Doha,25.29,51.53,La Villa Palace Hotel
196,Hervey Bay,-25.30,152.85,Oaks Hervey Bay Resort and Spa
316,Los Llanos de Aridane,28.66,-17.92,Hotel Hacienda de Abajo
359,Bocaiúva,-17.11,-43.81,Pontual Hotel
379,Dzaoudzi,-12.79,45.27,Oasis hôtel


In [31]:
Hotel_df.to_csv("../output_data/Hotel_Output.csv")

In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# # Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))